In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import mannwhitneyu

def preprocess_dataframe(df):
    """
    데이터프레임을 분석에 적합하도록 전처리합니다.
    (숫자형 변환, 공백 제거 등)
    """
    print("데이터 전처리를 시작합니다...")
    try:
        # 1. 숫자형(문자) 컬럼 정리
        numeric_object_cols = [
            '면적(m2)', '면적(평)', '수확량(kg)', '건중량(kg)',
            '평당 수확량', '건조 수확량(kg/10a)', '총 질소 살포량(kg/10a)'
        ]
        for col in numeric_object_cols:
            if col in df.columns:
                s = df[col].astype(str).str.replace(',', '', regex=False)
                s = s.str.replace(r'\s+', '', regex=True) # 모든 공백
                df[col] = pd.to_numeric(s, errors='coerce')

        # 2. 범주형(문자) 컬럼 정리
        categorical_cols = ['지역', '코드번호', '구분', 'CASE', '시비 처리', '작물', '품종']
        for col in categorical_cols:
            if col in df.columns:
                df[col] = df[col].astype(str).str.strip() # 앞뒤 공백
                if col == '품종':
                    # 품종은 중간 공백도 제거
                    df[col] = df[col].str.replace(r'\s+', '', regex=True)

        # 3. 년도 컬럼 숫자형 변환
        df['년도'] = pd.to_numeric(df['년도'], errors='coerce')

        # 4. 8대 토양 성분 숫자형 변환 (안정성)
        all_soil_components = [
            'soil_pH', 'soil_EC', 'soil_OM', 'soil_AVP', 'soil_AVSi', 'soil_K', 'soil_Ca', 'soil_Mg'
        ]
        for col in all_soil_components:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')

        print("데이터 전처리 완료.")
        return df

    except Exception as e:
        print(f"전처리 중 치명적 오류 발생: {e}")
        return None

def analysis_1_sales(df_clean):
    """
    [분석 1] '매출' 필지의 수확량/토양 상관관계 분석 및 시각화
    """
    print("\n--- [분석 1] '매출' 필지 상관관계 ---")
    try:
        df_sales = df_clean[df_clean['구분'] == '매출'].copy()

        corr_cols_1 = ['건조 수확량(kg/10a)', 'soil_OM', 'soil_AVSi']
        sales_data = df_sales[corr_cols_1].dropna()

        print(f"'매출' 구분 중 유효 데이터 {len(sales_data)}개로 상관관계 분석:")

        if len(sales_data) > 2:
            corr_matrix_1 = sales_data.corr()
            corr_results = corr_matrix_1['건조 수확량(kg/10a)']
            print(corr_results.to_string())

            # 시각화 (soil_AVSi vs 수확량)
            corr_avsi = corr_results.get('soil_AVSi', np.nan)

            fig = px.scatter(
                sales_data,
                x='soil_AVSi',
                y='건조 수확량(kg/10a)',
                size='soil_OM',
                trendline='ols',
                title=f"'매출' 필지: 유효규산(X) vs 수확량(Y) (크기: 유기물)<br>Corr(수확량, AVSi): {corr_avsi:.3f}",
                labels={
                    "건조 수확량(kg/10a)": "건조 수확량 (kg/10a)",
                    "soil_AVSi": "Soil 유효규산 (soil_AVSi)",
                    "soil_OM": "Soil 유기물 (soil_OM)"
                }
            )
            fig.update_layout(title_x=0.5, font=dict(family="Malgun Gothic, AppleGGothic, sans-serif"))
            fig.write_html("analysis_1_sales_scatter.html")
            print("→ 시각화 'analysis_1_sales_scatter.html' 저장 완료.")

            fig.show()

            image_filename = "매출필지 유효규산 수확량 크기.png"
            fig.write_image(
                image_filename,
                width=1200,
                height=600,
                scale=2  # 2배 해상도
            )
            print(f"\n성공: 그래프를 '{image_filename}' 파일로 저장했습니다.")


        else:
            print("'매출' 필지 상관관계 분석에 데이터가 부족합니다.")

    except Exception as e:
        print(f"분석 1 오류: {e}")

def analysis_2_sunchang(df_clean):
    """
    [분석 2] '순창' 지역 시비량 대비 수확량 증가율 (2025년) 분석 및 시각화
    """
    print("\n--- [분석 2] '순창' 지역 시비량 대비 수확량 증가율 (2025년) ---")
    try:
        df_sunchang_2025 = df_clean[(df_clean['지역'] == '순창') & (df_clean['년도'] == 2025)].copy()

        group_overdose_str = "밑거름 과다, 웃거름 변량"
        group_standard_str = "밑거름 정량, 웃거름 변량"
        cols_2 = ['건조 수확량(kg/10a)', '총 질소 살포량(kg/10a)']

        # --- [오류 수정 지점] ---
        # 집계 전, 해당 컬럼이 확실히 숫자형인지 다시 확인
        for col in cols_2:
            df_sunchang_2025[col] = pd.to_numeric(df_sunchang_2025[col], errors='coerce')
        # --- [오류 수정 완료] ---

        # 비교 대상 데이터만 필터링
        df_sunchang_compare = df_sunchang_2025[
            df_sunchang_2025['시비 처리'].isin([group_overdose_str, group_standard_str])
        ]

        # 각 그룹의 평균 계산
        stats_grouped = df_sunchang_compare.groupby('시비 처리')[cols_2].mean().reset_index()

        # 데이터가 없는 경우를 대비하여 stats_standard, stats_overdose 추출 시 안전장치 추가
        stats_standard = stats_grouped[stats_grouped['시비 처리'] == group_standard_str]
        stats_overdose = stats_grouped[stats_grouped['시비 처리'] == group_overdose_str]

        if stats_standard.empty or stats_overdose.empty:
            print("비교 대상 그룹('정량' 또는 '과다') 중 하나가 2025년 순창 데이터에 없습니다.")
            return

        stats_standard = stats_standard.iloc[0]
        stats_overdose = stats_overdose.iloc[0]

        print(f"'{group_standard_str}' (기준) 평균:\n{stats_standard[cols_2].to_string(float_format='%.2f')}\n")
        print(f"'{group_overdose_str}' (과다) 평균:\n{stats_overdose[cols_2].to_string(float_format='%.2f')}\n")

        # 비율 계산
        yield_over = stats_overdose['건조 수확량(kg/10a)']
        yield_std = stats_standard['건조 수확량(kg/10a)']
        n_over = stats_overdose['총 질소 살포량(kg/10a)']
        n_std = stats_standard['총 질소 살포량(kg/10a)']

        if pd.notna(yield_over) and pd.notna(yield_std) and yield_std != 0:
            yield_increase_pct = ((yield_over - yield_std) / yield_std) * 100
            print(f"→ 평균 수확량 증가율: {yield_increase_pct:.2f} %")
        else:
            print("→ 수확량 증가율 계산 불가")

        if pd.notna(n_over) and pd.notna(n_std) and n_std != 0:
            n_increase_pct = ((n_over - n_std) / n_std) * 100
            print(f"→ 총 질소 살포량 증가율: {n_increase_pct:.2f} %")
        else:
            print("→ 총 질소 살포량 증가율 계산 불가")

        # 시각화 (2개의 Y축을 가진 차트)
        fig = make_subplots(specs=[[{"secondary_y": True}]])

        # Y1: 수확량 막대
        fig.add_trace(
            go.Bar(
                x=stats_grouped['시비 처리'],
                y=stats_grouped['건조 수확량(kg/10a)'],
                name='평균 수확량 (kg/10a)',
                text=stats_grouped['건조 수확량(kg/10a)'].round(1),
                textposition='outside'
            ),
            secondary_y=False,
        )

        # Y2: 질소 살포량 선
        fig.add_trace(
            go.Scatter(
                x=stats_grouped['시비 처리'],
                y=stats_grouped['총 질소 살포량(kg/10a)'],
                name='평균 총 질소 살포량 (kg/10a)',
                mode='lines+markers+text',
                text=stats_grouped['총 질소 살포량(kg/10a)'].round(1),
                textposition='top center'
            ),
            secondary_y=True,
        )

        fig.update_layout(
            title_text="순창(2025): 시비량('과다' vs '정량') 대비 수확량 비교",
            title_x=0.5,
            font=dict(family="Malgun Gothic, AppleGGothic, sans-serif"),
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5)
        )
        # Y축 범위 자동 설정 (데이터에 맞게)
        fig.update_yaxes(title_text="평균 수확량 (kg/10a)", secondary_y=False, autorange=True, rangemode="tozero")
        fig.update_yaxes(title_text="평균 총 질소 살포량 (kg/10a)", secondary_y=True, autorange=True, rangemode="tozero")

        fig.show()

        image_filename = "순창(2025): 시비량('과다' vs '정량') 대비 수확량 비교.png"
        fig.write_image(
              image_filename,
              width=1200,
              height=600,
              scale=2  # 2배 해상도
            )
        print(f"\n성공: 그래프를 '{image_filename}' 파일로 저장했습니다.")

        fig.write_html("analysis_2_sunchang_comparison.html")
        print("→ 시각화 'analysis_2_sunchang_comparison.html' 저장 완료.")

    except Exception as e:
        print(f"분석 2 오류: {e}")

def analysis_3_gurye_sunchang(df_clean):
    """
    [분석 3] '구례' vs '순창' (새청무) 수확량/토양 비교 (2025년)
    """
    print("\n--- [분석 3] '구례' vs '순창' (새청무) 수확량/토양 비교 (2025년) ---")
    try:
        # 2025년에 '새청무'를 재배한 '구례', '순창' 데이터 필터링
        df_saecheongmu_2025 = df_clean[
            (df_clean['품종'] == '새청무') &
            (df_clean['년도'] == 2025) &
            (df_clean['지역'].isin(['구례', '순창']))
        ].copy()

        # [수정] 8대 토양 성분 모두 비교
        all_soil_components = ['soil_pH', 'soil_EC', 'soil_OM', 'soil_AVP', 'soil_AVSi', 'soil_K', 'soil_Ca', 'soil_Mg']
        comparison_cols = ['건조 수확량(kg/10a)'] + all_soil_components

        # 중앙값(Median)으로 비교
        comparison_stats = df_saecheongmu_2025.groupby('지역')[comparison_cols].median().transpose()

        print("\n[중앙값 비교표 (새청무, 2025년)]")
        print(comparison_stats.to_string(float_format="%.2f"))

        # '유효규산(soil_AVSi)' 통계 검증
        gurye_avsi = df_saecheongmu_2025[df_saecheongmu_2025['지역'] == '구례']['soil_AVSi'].dropna()
        sunchang_avsi = df_saecheongmu_2025[df_saecheongmu_2025['지역'] == '순창']['soil_AVSi'].dropna()

        if len(gurye_avsi) > 0 and len(sunchang_avsi) > 0:
            stat, p_value = mannwhitneyu(sunchang_avsi, gurye_avsi, alternative='greater')
            print(f"\n[유효규산 검증 (순창 > 구례)]")
            print(f"P-value: {p_value:.6f}")
            if p_value < 0.05:
                print("→ '순창'의 유효규산이 '구례'보다 통계적으로 유의미하게 높습니다. (p < 0.05)")
            else:
                print("→ '순창'의 유효규산이 '구례'보다 높지만, 통계적으로 유의미한 차이는 아닙니다. (p >= 0.05)")
        else:
            print("\n[유효규산 검증] 비교할 데이터가 부족합니다.")

        # 시각화 (Box Plot) - 8대 성분 + 수확량
        plot_cols_3 = ['건조 수확량(kg/10a)'] + all_soil_components
        melted_data_3 = df_saecheongmu_2025.melt(
            id_vars=['지역', '코드번호'],
            value_vars=plot_cols_3,
            var_name='측정 항목',
            value_name='값'
        )
        melted_data_3 = melted_data_3.dropna(subset=['값'])

        fig = px.box(
            melted_data_3,
            x='지역',
            y='값',
            color='지역',
            facet_row='측정 항목',
            points='all',
            title='구례 vs 순창 (새청무, 2025년) 수확량 및 전체 토양 비교',
            labels={"값": "측정 값", "지역": "지역"}
        )
        fig.update_layout(height=2000, title_x=0.5, font=dict(family="Malgun Gothic, AppleGGothic, sans-serif"), showlegend=False)
        fig.update_yaxes(matches=None) # 각 차트 Y축 독립

        fig.show()

        image_filename = "구례 vs 순창 (새청무, 2025년) 수확량 및 전체 토양 비교.png"
        fig.write_image(
              image_filename,
              width=1200,
              height=600,
              scale=2  # 2배 해상도
            )
        print(f"\n성공: 그래프를 '{image_filename}' 파일로 저장했습니다.")



        fig.write_html("analysis_3_gurye_sunchang_boxplot.html")
        print("→ 시각화 'analysis_3_gurye_sunchang_boxplot.html' 저장 완료.")

    except Exception as e:
        print(f"분석 3 오류: {e}")

# --- 메인 코드 실행 ---
if __name__ == "__main__":
    try:
        print("--- [시작] 최신 데이터 파일 로드 ---")
        # 이 스크립트가 실행되는 환경에 파일이 있어야 합니다.
        df_main = pd.read_csv("data/25년_수확량 통계.csv")

        # 1. 공통 전처리
        df_clean = preprocess_dataframe(df_main)

        if df_clean is not None:
            # 2. 개별 분석 실행
            analysis_1_sales(df_clean.copy())
            analysis_2_sunchang(df_clean.copy())
            analysis_3_gurye_sunchang(df_clean.copy())

        print("\n--- [완료] 모든 분석 및 시각화 파일 생성이 완료되었습니다. ---")

    except FileNotFoundError:
        print("오류: '25년_수확량 통계.csv' 파일을 찾을 수 없습니다.")
    except Exception as e:
        print(f"메인 실행 중 오류 발생: {e}")

--- [시작] 최신 데이터 파일 로드 ---
데이터 전처리를 시작합니다...
데이터 전처리 완료.

--- [분석 1] '매출' 필지 상관관계 ---
'매출' 구분 중 유효 데이터 28개로 상관관계 분석:
건조 수확량(kg/10a)    1.000000
soil_OM           0.306483
soil_AVSi         0.652483
→ 시각화 'analysis_1_sales_scatter.html' 저장 완료.



성공: 그래프를 '매출필지 유효규산 수확량 크기.png' 파일로 저장했습니다.

--- [분석 2] '순창' 지역 시비량 대비 수확량 증가율 (2025년) ---
분석 2 오류: 'str' object is not callable

--- [분석 3] '구례' vs '순창' (새청무) 수확량/토양 비교 (2025년) ---

[중앙값 비교표 (새청무, 2025년)]
지역                 구례     순창
건조 수확량(kg/10a) 590.80 869.42
soil_pH           NaN    NaN
soil_EC           NaN    NaN
soil_OM         37.36  55.06
soil_AVP          NaN    NaN
soil_AVSi      172.95 864.85
soil_K            NaN    NaN
soil_Ca           NaN    NaN
soil_Mg           NaN    NaN

[유효규산 검증 (순창 > 구례)]
P-value: 0.000044
→ '순창'의 유효규산이 '구례'보다 통계적으로 유의미하게 높습니다. (p < 0.05)



성공: 그래프를 '구례 vs 순창 (새청무, 2025년) 수확량 및 전체 토양 비교.png' 파일로 저장했습니다.
→ 시각화 'analysis_3_gurye_sunchang_boxplot.html' 저장 완료.

--- [완료] 모든 분석 및 시각화 파일 생성이 완료되었습니다. ---


In [4]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 45 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   지역                 88 non-null     object 
 1   년도                 88 non-null     int64  
 2   코드번호               88 non-null     object 
 3   주소                 88 non-null     object 
 4   경작자                88 non-null     object 
 5   면적(m2)             68 non-null     float64
 6   면적(평)              86 non-null     float64
 7   구분                 88 non-null     object 
 8   CASE               88 non-null     object 
 9   시비 처리              88 non-null     object 
 10   작물                88 non-null     object 
 11  품종                 88 non-null     object 
 12  soil_pH            58 non-null     float64
 13  soil_EC            52 non-null     float64
 14  soil_OM            80 non-null     float64
 15  soil_AVP           58 non-null     float64
 16  soil_AVSi          80 non-nu